In [6]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

In [11]:
import pandas as pd

In [2]:
patch_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'

In [12]:
for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:

    print stack
# stack = 'MD585'
# stack = 'MD594'
# stack = 'MD589'

# first_bs_sec, last_bs_sec = section_range_lookup[stack]

    box_x, box_y, box_w, box_h = detect_bbox_lookup[stack]
    first, last = detect_bbox_range_lookup[stack]

    patch_size = 224
    half_size = patch_size/2
    stride = 56

    dm = DataManager(stack=stack)

    ys, xs = np.meshgrid(np.arange(half_size, dm.image_height - half_size, stride), 
                         np.arange(half_size, dm.image_width - half_size, stride),
                         indexing='xy')

    sample_locations = np.c_[xs.flat, ys.flat]


    xmin = max(half_size, box_x*32)
    xmax = min(dm.image_width-half_size-1, (box_x+box_w)*32)
    ymin = max(half_size, box_y*32)
    ymax = min(dm.image_height-half_size-1, (box_y+box_h)*32)

    ######################

    indices_roi_allSections = defaultdict(dict)

    for sec in range(first, last+1):

        dm.set_slice(sec)

        mask = dm.load_thumbnail_mask()
        indices_fg = np.where(mask[sample_locations[:,1]/32, sample_locations[:,0]/32])[0]
        indices_bg = np.setdiff1d(range(sample_locations.shape[0]), indices_fg)

        indices_roi = np.where(np.all(np.c_[sample_locations[:,0] > xmin, sample_locations[:,1] > ymin, 
                                            sample_locations[:,0] < xmax, sample_locations[:,1] < ymax], axis=1))[0]

        indices_roi = np.setdiff1d(indices_roi, indices_bg)
        print len(indices_roi), 'patches in ROI'

        indices_roi_allSections[sec]['roi1'] = indices_roi


    indices_roi_allSections_df = pd.DataFrame(indices_roi_allSections)

    # indices_roi_allSections_df

    dump_path = os.path.join(patch_dir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})

    indices_roi_allSections_df.to_hdf(dump_path, 'indices_allROIs_allSections')
    pd.Series([patch_size, stride, dm.image_width, dm.image_height]).to_hdf(dump_path, 'grid_parameters')

MD585
20979 patches in ROI
21122 patches in ROI
20822 patches in ROI
21384 patches in ROI
21489 patches in ROI
21762 patches in ROI
21975 patches in ROI
22145 patches in ROI
22228 patches in ROI
22235 patches in ROI
22468 patches in ROI
22452 patches in ROI
22607 patches in ROI
22774 patches in ROI
20789 patches in ROI
22525 patches in ROI
23051 patches in ROI
23011 patches in ROI
23008 patches in ROI
23392 patches in ROI
23061 patches in ROI
22866 patches in ROI
22233 patches in ROI
23433 patches in ROI
23468 patches in ROI
23709 patches in ROI
23257 patches in ROI
23619 patches in ROI
23641 patches in ROI
23815 patches in ROI
23669 patches in ROI
23778 patches in ROI
23672 patches in ROI
23773 patches in ROI
23732 patches in ROI
23841 patches in ROI
23820 patches in ROI
23824 patches in ROI
22813 patches in ROI
23642 patches in ROI
23930 patches in ROI
24012 patches in ROI
24211 patches in ROI
24136 patches in ROI
24160 patches in ROI
23935 patches in ROI
24183 patches in ROI
24074 p

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 2

 patches in ROI
MD592
21416 patches in ROI
20566 patches in ROI
21777 patches in ROI
21659 patches in ROI
21909 patches in ROI
22673 patches in ROI
22319 patches in ROI
21480 patches in ROI
22432 patches in ROI
21964 patches in ROI
22848 patches in ROI
22836 patches in ROI
23297 patches in ROI
23261 patches in ROI
23392 patches in ROI
22672 patches in ROI
23864 patches in ROI
23669 patches in ROI
25328 patches in ROI
24229 patches in ROI
24675 patches in ROI
24312 patches in ROI
24745 patches in ROI
24513 patches in ROI
25071 patches in ROI
24694 patches in ROI
24476 patches in ROI
25070 patches in ROI
25658 patches in ROI
25313 patches in ROI
25781 patches in ROI
25274 patches in ROI
26097 patches in ROI
24981 patches in ROI
26350 patches in ROI
25680 patches in ROI
26342 patches in ROI
26473 patches in ROI
25705 patches in ROI
25832 patches in ROI
26166 patches in ROI
25975 patches in ROI
26473 patches in ROI
26531 patches in ROI
26879 patches in ROI
26972 patches in ROI
26355 patche

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 2

 patches in ROI
MD591
26093 patches in ROI
26046 patches in ROI
26168 patches in ROI
26339 patches in ROI
26975 patches in ROI
27166 patches in ROI
27799 patches in ROI
27024 patches in ROI
27362 patches in ROI
27611 patches in ROI
28027 patches in ROI
28076 patches in ROI
28385 patches in ROI
28622 patches in ROI
28743 patches in ROI
28999 patches in ROI
29356 patches in ROI
29630 patches in ROI
29322 patches in ROI
29537 patches in ROI
30008 patches in ROI
29868 patches in ROI
29925 patches in ROI
29951 patches in ROI
30520 patches in ROI
30315 patches in ROI
30814 patches in ROI
30522 patches in ROI
30548 patches in ROI
31014 patches in ROI
30813 patches in ROI
30954 patches in ROI
31023 patches in ROI
31888 patches in ROI
32035 patches in ROI
31638 patches in ROI
31526 patches in ROI
31984 patches in ROI
31969 patches in ROI
31333 patches in ROI
31277 patches in ROI
31769 patches in ROI
31540 patches in ROI
31754 patches in ROI
31252 patches in ROI
31430 patches in ROI
31670 patche

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 2

 patches in ROI
MD598
16638 patches in ROI
16900 patches in ROI
17073 patches in ROI
17135 patches in ROI
17245 patches in ROI
17334 patches in ROI
17652 patches in ROI
17759 patches in ROI
17810 patches in ROI
17882 patches in ROI
18148 patches in ROI
18156 patches in ROI
18537 patches in ROI
18518 patches in ROI
18757 patches in ROI
18862 patches in ROI
18983 patches in ROI
18977 patches in ROI
19165 patches in ROI
19608 patches in ROI
19751 patches in ROI
19811 patches in ROI
19824 patches in ROI
20199 patches in ROI
20358 patches in ROI
20430 patches in ROI
20652 patches in ROI
20115 patches in ROI
20248 patches in ROI
21136 patches in ROI
20898 patches in ROI
21339 patches in ROI
21111 patches in ROI
20449 patches in ROI
21859 patches in ROI
21632 patches in ROI
21636 patches in ROI
21863 patches in ROI
21796 patches in ROI
21867 patches in ROI
22027 patches in ROI
22237 patches in ROI
22109 patches in ROI
22189 patches in ROI
22492 patches in ROI
22367 patches in ROI
22467 patche

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 2

 patches in ROI
